# Create Network

In [1]:
import pandas as pd
f = '../data/formatted-courses.json'

In [2]:
df = pd.read_json(f)
print (df.columns)
df.head()

Index(['Description', 'Link', 'college', 'courseID', 'course_num',
       'department', 'hrs', 'name', 'prereqs'],
      dtype='object')


,Description,Link,college,courseID,course_num,department,hrs,name,prereqs
0,This course offers an examination of foundatio...,https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:3000,3000,EPLS,3.0,Foundations of Education,[]
1,"The focus of this course, which is required fo...",https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:4180,4180,EPLS,3.0,Human Relations for the Classroom Teachr,[]
2,Certificate for Interscholastic Athletics/Acti...,https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:4200,4200,EPLS,3.0,Diversity and Inclusion in Athletics,[]
3,This section is offered through Distance and O...,https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:5090,5090,EPLS,3.0,Instr Coaching for Teaching Excellence,[]
4,"Current selected functions, issues, policies o...",https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:5100,5100,EPLS,3.0,Issues and Policies in Higher Education,[]


### Prereqs

- remove duplicates
- add to prereqs list

In [3]:
prereqs_list = []
def make_prereq(course, prereqs):
    for i in prereqs:
        prereqs_list.append({"source": i, "target": course, "value": 1})

# remove dupes
df['prereqs'] = df['prereqs'].apply(lambda x: list(set(x)))

# create prereqs links
df.apply(lambda x: make_prereq(x['courseID'], x['prereqs']), axis=1)


0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
3452    None
3453    None
3454    None
3455    None
3456    None
3457    None
3458    None
3459    None
3460    None
3461    None
3462    None
3463    None
3464    None
3465    None
3466    None
3467    None
3468    None
3469    None
3470    None
3471    None
3472    None
3473    None
3474    None
3475    None
3476    None
3477    None
3478    None
3479    None
3480    None
3481    None
dtype: object

### Antecedents

*list of courses that a given course leads to.*

- flip each source and add it to a list

In [4]:
antecedents_list = []
for i in prereqs_list:
    flipped = {"source": i["target"], "target": i["source"], "value": 1}
    antecedents_list.append(flipped)

In [5]:
indexed_df = df.set_index('courseID', drop=False)
indexed_df.head()

,Description,Link,college,courseID,course_num,department,hrs,name,prereqs
courseID,,,,,,,,,
EPLS:3000,This course offers an examination of foundatio...,https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:3000,3000,EPLS,3.0,Foundations of Education,[]
EPLS:4180,"The focus of this course, which is required fo...",https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:4180,4180,EPLS,3.0,Human Relations for the Classroom Teachr,[]
EPLS:4200,Certificate for Interscholastic Athletics/Acti...,https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:4200,4200,EPLS,3.0,Diversity and Inclusion in Athletics,[]
EPLS:5090,This section is offered through Distance and O...,https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:5090,5090,EPLS,3.0,Instr Coaching for Teaching Excellence,[]
EPLS:5100,"Current selected functions, issues, policies o...",https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:5100,5100,EPLS,3.0,Issues and Policies in Higher Education,[]


In [6]:
# create a map
courseIDs = indexed_df['courseID'].unique().tolist()
course_prereqs = {}
for i in courseIDs:
    course_prereqs[i] = []

# populate map
for i in prereqs_list:
    course = course_prereqs[i['target']]
    course.append(i)
    
# move the data to the correct row in DF
indexed_df['prereqs'] = indexed_df['courseID'].apply(lambda x: course_prereqs[x])

In [7]:
course_after = {}
for i in courseIDs:
    course_after[i] = []
    
for i in antecedents_list:
    course = course_after[i['source']]
    course.append(i)
    
indexed_df['after'] = indexed_df['courseID'].apply(lambda x: course_after[x])

In [8]:
indexed_df.columns

Index(['Description', 'Link', 'college', 'courseID', 'course_num',
       'department', 'hrs', 'name', 'prereqs', 'after'],
      dtype='object')

In [9]:
indexed_df[['name', 'prereqs', 'after']].iloc[400:500]

,name,prereqs,after
courseID,,,
CW:3218,Creative Writing for New Media,[],[]
CW:3870,Advanced Fiction Writing,"[{'source': 'CW:2870', 'target': 'CW:3870', 'v...","[{'source': 'CW:3870', 'target': 'CW:2870', 'v..."
CW:3875,Advanced Poetry Writing,"[{'source': 'CW:2875', 'target': 'CW:3875', 'v...","[{'source': 'CW:3875', 'target': 'CW:2875', 'v..."
CW:4745,The Sentence: Strategies for Writing,[],[]
CW:4751,Creative Writing for the Musician,[],[]
CW:4760,The Art of Revision: Writing for Clarity,[],[]
CW:4870,Undergraduate Writers' Workshop: Fiction,[],[]
CW:4875,Undergraduate Writers' Workshop: Poetry,[],[]
CW:4894,Undergrad Project in Creative Writing,[],[]


In [10]:
indexed_df.rename(columns={'Description': 'description', 'Link': 'link'})

,description,link,college,courseID,course_num,department,hrs,name,prereqs,after
courseID,,,,,,,,,,
EPLS:3000,This course offers an examination of foundatio...,https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:3000,3000,EPLS,3.0,Foundations of Education,[],[]
EPLS:4180,"The focus of this course, which is required fo...",https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:4180,4180,EPLS,3.0,Human Relations for the Classroom Teachr,[],[]
EPLS:4200,Certificate for Interscholastic Athletics/Acti...,https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:4200,4200,EPLS,3.0,Diversity and Inclusion in Athletics,[],[]
EPLS:5090,This section is offered through Distance and O...,https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:5090,5090,EPLS,3.0,Instr Coaching for Teaching Excellence,[],[]
EPLS:5100,"Current selected functions, issues, policies o...",https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:5100,5100,EPLS,3.0,Issues and Policies in Higher Education,[],[]
EPLS:5102,None,https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:5102,5102,EPLS,3.0,History of American Education,[],[]
EPLS:5157,None,https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:5157,5157,EPLS,3.0,Ethics in Education,[],[]
EPLS:5240,"In this course, students will gain coding and ...",https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:5240,5240,EPLS,1.0,Topics in Education ...,[],[]
EPLS:5240,This course explores the history of African Am...,https://myui.uiowa.edu/my-ui/courses/details.p...,College of Education,EPLS:5240,5240,EPLS,3.0,Topics in Education ...,[],[]


In [13]:
def make_network(a_list):
    if (len(a_list) < 1):
        return
    
    nodes = []
    before = []
    after = []
    
    for i in a_list:
        course = indexed_df.loc[i]
        for i in course['prereqs']:
            prereq = indexed_df.loc[i['source']]
            nodes.append(prereq)
            before.append(i)
        for i in course['after']:
            antecedent = indexed_df.loc[i['target']]
            nodes.append(antecedent)
            after.append(i)
    return {"nodes": nodes, "before": before, "after": after}

In [14]:
make_network(['CW:3870'])

{'after': [{'source': 'CW:3870', 'target': 'CW:2870', 'value': 1}],
 'before': [{'source': 'CW:2870', 'target': 'CW:3870', 'value': 1}],
 'nodes': [Description    Writing fiction is a craft and a process.  Stu...
  Link           https://myui.uiowa.edu/my-ui/courses/details.p...
  college                     College of Liberal Arts and Sciences
  courseID                                                 CW:2870
  course_num                                                  2870
  department                                                    CW
  hrs                                                            3
  name                                             Fiction Writing
  prereqs                                                       []
  after                                                         []
  Name: CW:2870, dtype: object,
  Description    Writing fiction is a craft and a process.  Stu...
  Link           https://myui.uiowa.edu/my-ui/courses/details.p...
  college         